# Punto # 1

#### Cargas las librerias necesarias

In [30]:
import pandas as pd  # Librería para manipulación y análisis de datos
import numpy as np  # Librería para operaciones numéricas y generación de datos aleatorios
import os  # Librería para interactuar con el sistema de archivos

### General funciones

Función para crear un dataframe (`df`) con tres variables:  
- **Producto** (`str`)  
- **Ventas** (`float`)  
- **Categoría** (`category`)  

Debe contener **1000 observaciones** con las siguientes características:

| Index | Producto | Ventas | Categoría |
|-------|---------|--------|-----------|
| 0     | P0001   | 1500   | GIMPAR    |
| 1     | P0002   | 3000   | GPAR      |
| 2     | P0003   | 1900   | GIMPAR    |
| ...   | ...     | ...    | ...       |
| 999   | P1000   | 6252   | GPAR      |

**Condiciones**:
- `Index` es el índice generado por `pandas` u otra biblioteca de manejo de datos.
- `Producto` debe estar ordenado del 1 al 1000, con formato de cuatro dígitos (`P0001`, `P0002`, ...).
- `Ventas` debe ser un número aleatorio entre **1000 y 10000**.
- `Categoría`:  
  - Si el número del producto es **impar**, será `"GIMPAR"`.  
  - Si es **par**, será `"GPAR"`.

In [31]:
# Definir función para generar el DataFrame con las condiciones establecidas
def generar_dataframe():
    # Establecer la semilla para la reproducibilidad
    np.random.seed(42)
    
    # Crear lista de productos con formato P0001, P0002, ..., P1000
    productos = [f'P{i:04d}' for i in range(1, 1001)]
    
    # Generar valores aleatorios de ventas entre 1000 y 10000
    ventas = np.random.randint(1000, 10001, size=1000)
    
    # Asignar categorías según la paridad del número del producto
    categorias = ['GIMPAR' if int(p[1:]) % 2 != 0 else 'GPAR' for p in productos]
    
    # Crear DataFrame inicial
    df = pd.DataFrame({
        'Producto': productos,
        'Ventas': ventas.astype(float),  # Asegurar tipo float
        'Categoría': pd.Categorical(categorias)
    })

    return df

Función para la creación de la variable "Venta Neta"  
Se debe generar una nueva variable **"Venta Neta"**, calculada de la siguiente manera:  


$\text{Venta Neta} = \text{Ventas} - \text{(Impuesto IGV 18\%)} - \text{(Descuento variable)}$

**Tabla de descuentos**:

| Rango de Ventas | Descuento |
|-----------------|-----------|
| 1000 - 1500    | 1%        |
| 1500 - 3000    | 2%        |
| 3000 - 7000    | 3%        |
| 7000 - 9000    | 4%        |
| 10000 >        | 5%        |

In [32]:
# Definir función para calcular Venta Neta
def calcular_venta_neta(df):
    # Calcular el IGV (18% de las ventas)
    df['IGV'] = df['Ventas'] * 0.18

    # Definir función para calcular descuento según el rango de ventas
    def calcular_descuento(venta):
        if 1000 <= venta < 1500:
            return venta * 0.01  # 1%
        elif 1500 <= venta < 3000:
            return venta * 0.02  # 2%
        elif 3000 <= venta < 7000:
            return venta * 0.03  # 3%
        elif 7000 <= venta < 9000:
            return venta * 0.04  # 4%
        else:
            return venta * 0.05  # 5%
    
    # Aplicar la función de descuento
    df['Descuento'] = df['Ventas'].apply(calcular_descuento)
    
    # Calcular la Venta Neta
    df['Venta Neta'] = df['Ventas'] - df['IGV'] - df['Descuento']
    
    # Eliminar las columnas intermedias si no son necesarias
    df.drop(columns=['IGV', 'Descuento'], inplace=True)

    return df

Aplicar funciones y verficar el codntenido del df.

In [33]:
# Generar el DataFrame y calcular la Venta Neta
df = generar_dataframe()
df = calcular_venta_neta(df)
df.head(5)

,Producto,Ventas,Categoría,Venta Neta
0,P0001,8270.0,GIMPAR,6450.60
1,P0002,1860.0,GPAR,1488.00
2,P0003,6390.0,GIMPAR,5048.10
3,P0004,6191.0,GPAR,4890.89
4,P0005,6734.0,GIMPAR,5319.86


Guardamos el DataFrame en un archivo CSV para posibles analisis futuros

In [35]:
# Exportar el DataFrame a un archivo CSV
df.to_csv('ventas_netas.csv', index=False, encoding='utf-8')
print("El DataFrame ha sido guardado en 'ventas_netas.csv' exitosamente.")

El DataFrame ha sido guardado en 'ventas_netas.csv' exitosamente.


# Punto # 2

In [28]:
# Guardar el DataFrame resultante en un archivo Parquet con compresión Snappy
df = generar_dataframe() # Generar el DataFrame original
df.to_parquet('ventas_netas.parquet', index=False, compression='snappy')
print("El DataFrame ha sido guardado en 'ventas_netas.parquet' con compresión Snappy.")

El DataFrame ha sido guardado en 'ventas_netas.parquet' con compresión Snappy.


# Punto # 3

### Pasar hojas de calculo a csv para mayor comodidad.

In [ ]:

# Mostrar mensaje introductorio
print("Este notebook extrae todas las hojas de un archivo Excel y las guarda como archivos CSV.")

# Definir el nombre del archivo Excel
excel_file = "BASES_MODELO(2).xlsx"

# Definir el nombre de la carpeta donde se guardarán los CSV
output_folder = "hojas_csv"

# Crear la carpeta si no existe
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"Carpeta '{output_folder}' creada exitosamente.")
else:
    print(f"La carpeta '{output_folder}' ya existe.")

# Cargar el archivo Excel
xls = pd.ExcelFile(excel_file)
print(f"El archivo '{excel_file}' ha sido cargado exitosamente.")

# Iterar sobre cada hoja y guardarla en CSV
for sheet_name in xls.sheet_names:
    print(f"Procesando la hoja: {sheet_name}")
    
    # Leer la hoja
    df = xls.parse(sheet_name)
    
    # Nombre del archivo CSV basado en la hoja
    csv_file = os.path.join(output_folder, f"{sheet_name}.csv")
    
    # Guardar como CSV sin incluir el índice
    df.to_csv(csv_file, index=False, encoding="utf-8")
    
    print(f"Hoja '{sheet_name}' guardada en {csv_file}")

print("Proceso completado. Todas las hojas han sido exportadas como CSV.")

Este notebook extrae todas las hojas de un archivo Excel y las guarda como archivos CSV.
La carpeta 'hojas_csv' ya existe.
El archivo 'BASES_MODELO(2).xlsx' ha sido cargado exitosamente.
Procesando la hoja: DESCRIPICION
Hoja 'DESCRIPICION' guardada en hojas_csv/DESCRIPICION.csv
Procesando la hoja: BASE_VARIABLES
Hoja 'BASE_VARIABLES' guardada en hojas_csv/BASE_VARIABLES.csv
Procesando la hoja: BASE_ASESORA
Hoja 'BASE_ASESORA' guardada en hojas_csv/BASE_ASESORA.csv
Procesando la hoja: BASE_ESTIMADOS
Hoja 'BASE_ESTIMADOS' guardada en hojas_csv/BASE_ESTIMADOS.csv
Proceso completado. Todas las hojas han sido exportadas como CSV.
